In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../data/fusionada.csv')

## Filtro

### Trabajando columna **'recommend'**

In [7]:
# Cargar el DataFrame
columnas = ['item_id', 'clasificacion', 'playtime_forever', 'recommend']
item = pd.read_csv('../data/fusionada.csv', usecols=columnas)
item.head()

,item_id,clasificacion,playtime_forever,recommend
0,282010,Action,466.0,True
1,70,Action,1395.0,True
2,70,Action,590.0,True
3,70,Action,5599.0,True
4,70,Action,64.0,True


In [8]:
# Agrupamos por interacciones 'item_id' y contamos las 'recommed' para cada juego
frecuencia_juegos = item[item['recommend'] == True]['item_id'].value_counts().reset_index()
frecuencia_juegos.columns = ['item_id', 'frecuencia_rec']
frecuencia_juegos

,item_id,frecuencia_rec
0,730,3119
1,4000,1491
2,304930,849
3,218620,775
4,550,639
...,...,...
2459,457580,1
2460,446120,1
2461,454320,1
2462,457520,1


In [9]:
# Unimos el dataframe principal con el DF 'frecuencia de juegos'
df_merge1 = pd.merge(item, frecuencia_juegos, on='item_id', how='left')
df_merge1['frecuencia_rec'] = df_merge1['frecuencia_rec'].fillna(0)
df_merge1

,item_id,clasificacion,playtime_forever,recommend,frecuencia_rec
0,282010,Action,466.0,True,1.0
1,70,Action,1395.0,True,52.0
2,70,Action,590.0,True,52.0
3,70,Action,5599.0,True,52.0
4,70,Action,64.0,True,52.0
...,...,...,...,...,...
40010,80,Action,2416.0,False,8.0
40011,80,Action,15.0,False,8.0
40012,80,Action,442.0,True,8.0
40013,80,Action,72.0,True,8.0


In [10]:
# Convertir la columna 'frecuencia_rec' a tipo de dato entero.
df_merge1['frecuencia_rec'] = df_merge1['frecuencia_rec'].astype(int)
df_merge1.head()

,item_id,clasificacion,playtime_forever,recommend,frecuencia_rec
0,282010,Action,466.0,True,1
1,70,Action,1395.0,True,52
2,70,Action,590.0,True,52
3,70,Action,5599.0,True,52
4,70,Action,64.0,True,52


In [11]:
df_merge1['recommend'].value_counts()

recommend
True     35843
False     4172
Name: count, dtype: int64

Filtrar datos relevantes en df_merge1 basándose en la frecuencia de recomendación

In [12]:
filtro_rec = df_merge1[df_merge1['frecuencia_rec'] >= 5]
filtro_rec

,item_id,clasificacion,playtime_forever,recommend,frecuencia_rec
1,70,Action,1395.0,True,52
2,70,Action,590.0,True,52
3,70,Action,5599.0,True,52
4,70,Action,64.0,True,52
5,70,Action,3235.0,True,52
...,...,...,...,...,...
40010,80,Action,2416.0,False,8
40011,80,Action,15.0,False,8
40012,80,Action,442.0,True,8
40013,80,Action,72.0,True,8


In [13]:
# Verificamos la cantidad de 'item_id' tenemos para trabajar
filtro_rec['item_id'].nunique()

802

In [15]:
# Seleccionar columnas relevantes en df_f_rec para análisis posterior
columnas = ['item_id', 'clasificacion', 'frecuencia_rec']
df_f_rec = filtro_rec[columnas]

In [16]:
# Eliminar duplicados manteniendo la primera ocurrencia
df_f_rec_agrupado = df_f_rec.drop_duplicates(subset=['item_id'])
df_f_rec_agrupado

,item_id,clasificacion,frecuencia_rec
1,70,Action,52
54,2400,Action,21
77,1520,Indie,5
86,4000,Indie,1491
1619,2990,Racing,6
...,...,...,...
39935,60,Action,8
39943,10,Action,44
39989,130,Action,5
39998,13230,Action,6


Terminamos el trabajo de la columna **recommend**, consiguiendo disminuir a 802 juegos **'item_id'**, este filtro nos facilitará el proceso de ML

### Trabajo columna **'playtime_forever'**

In [19]:
# Cargar el DataFrame
columnas = ['item_id', 'clasificacion', 'playtime_forever', 'recommend']
item2 = pd.read_csv('../data/fusionada.csv', usecols=columnas)
item2.head()

,item_id,clasificacion,playtime_forever,recommend
0,282010,Action,466.0,True
1,70,Action,1395.0,True
2,70,Action,590.0,True
3,70,Action,5599.0,True
4,70,Action,64.0,True


In [20]:
# Sumar el tiempo de juego para cada juego (item_id)
playtime = item2.groupby('item_id')['playtime_forever'].sum().reset_index()
playtime.columns = ['item_id', 'total_playtime']
playtime


,item_id,total_playtime
0,10,747975.0
1,20,72492.0
2,30,2392.0
3,40,145.0
4,50,1234.0
...,...,...
2717,521340,5.0
2718,521430,5.0
2719,521570,79.0
2720,521990,16.0


In [21]:
# Consultando la cantidad de juegos tenemos 'item_id'
playtime['item_id'].nunique()

2722

In [22]:
# Ordenar por la columna 'playtime_forever' de manera ascendente
playtime_ord = playtime.sort_values(by='total_playtime')
playtime_ord.head(10)

,item_id,total_playtime
2173,362280,1.0
401,45710,1.0
2143,357380,1.0
104,7760,1.0
1901,328830,5.0
520,96200,5.0
2458,407600,5.0
1722,307990,5.0
1840,321610,5.0
2232,369060,5.0


In [23]:
# Al verificar que nuestra columna total_playtime tiene datos booleanos, los cambiamos a enteros, para optimizar los futuros procesos.
playtime_ord['total_playtime'] = playtime_ord['total_playtime'].astype(int)

In [24]:
#Filtrar juegos con tiempo de juego superior a 2000 minutos
filtro_time = playtime_ord[playtime_ord['total_playtime'] >= 2000]
filtro_time

,item_id,total_playtime
1267,262490,2000
1537,289600,2001
2263,374040,2027
176,12590,2028
2499,413420,2031
...,...,...
552,105600,10320781
892,230410,12321959
771,218620,12546425
68,4000,49464882


Luego de trabajar con **'playtime_forever**  hemos logrado identificar a los datos de juegos más optimos para los siguientes procesos

In [29]:
# Crear un DataFrame con las columnas necesarias
columnas = ['item_id', 'clasificacion']
df_genres = df[columnas]

frecuencia_genres = df_genres.groupby(['item_id', 'clasificacion']).size().reset_index(name='frecuencia_genres')

# Consolidar la información agrupando por item_id
df_genres_agrupado = frecuencia_genres.groupby('item_id', as_index=False).agg({'clasificacion': ' | '.join, 'frecuencia_genres': 'sum'})
df_genres_agrupado

,item_id,clasificacion,frecuencia_genres
0,10,Action,45
1,20,Action,15
2,30,Action,4
3,40,Action,1
4,50,Action,3
...,...,...,...
2717,521340,Casual,1
2718,521430,Indie,1
2719,521570,Casual,2
2720,521990,Action,1


In [30]:
# Verificamos la cantidad de juegos que entrarían a futuros proceso ML
df_genres_agrupado['item_id'].nunique()

2722

In [31]:
filtro_genres= df_genres_agrupado[df_genres_agrupado['frecuencia_genres'] >= 5]
filtro_genres

,item_id,clasificacion,frecuencia_genres
0,10,Action,45
1,20,Action,15
5,60,Action,8
6,70,Action,53
7,80,Action,11
...,...,...,...
2611,444090,Action,13
2630,449140,Indie,5
2649,459820,Casual,5
2654,461560,Adventure,5


In [32]:
# Verificamos nuevamente como ha cambiado la cantidad de juegos
filtro_genres['item_id'].nunique()

880

Luego de trabajar con **'frecuencia_genres'**  hemos logrado identificar 880 juegos con mejor data, para nuestros siguientes proceso

## Guardar DF que servirá para realizar el proceso de ML recomendación 'item'-'item'

In [33]:
# Unir los DataFrames
df_final = pd.merge(df_f_rec_agrupado, filtro_time, on='item_id')
df_final = pd.merge(df_final, filtro_genres, on='item_id')
df_final

,item_id,clasificacion_x,frecuencia_rec,total_playtime,clasificacion_y,frecuencia_genres
0,70,Action,52,56884,Action,53
1,2400,Action,21,10608,Action,22
2,1520,Indie,5,17155,Indie,7
3,4000,Indie,1491,49464882,Indie,1533
4,2990,Racing,6,5933,Racing,6
...,...,...,...,...,...,...
717,1510,Indie,5,5666,Indie,5
718,20,Action,10,72492,Action,15
719,10,Action,44,747975,Action,45
720,13230,Action,6,6616,Action,6


In [34]:
# Supongamos que quieres renombrar las columnas 'genres_x' y 'genres_y'
df_final = df_final.rename(columns={'frecuencia_genres': 'genres', 'frecuencia_rec': 'recommend','total_playtime':'playtime_forever'})
df_final

,item_id,clasificacion_x,recommend,playtime_forever,clasificacion_y,genres
0,70,Action,52,56884,Action,53
1,2400,Action,21,10608,Action,22
2,1520,Indie,5,17155,Indie,7
3,4000,Indie,1491,49464882,Indie,1533
4,2990,Racing,6,5933,Racing,6
...,...,...,...,...,...,...
717,1510,Indie,5,5666,Indie,5
718,20,Action,10,72492,Action,15
719,10,Action,44,747975,Action,45
720,13230,Action,6,6616,Action,6


In [35]:
columnas = ['item_id', 'genres', 'recommend', 'playtime_forever']
df_final = df_final[columnas]
df_final

,item_id,genres,recommend,playtime_forever
0,70,53,52,56884
1,2400,22,21,10608
2,1520,7,5,17155
3,4000,1533,1491,49464882
4,2990,6,6,5933
...,...,...,...,...
717,1510,5,5,5666
718,20,15,10,72492
719,10,45,44,747975
720,13230,6,6,6616


In [37]:
# Mergeamos con la data original, usando 'item_id'. Gracias al trabajo realizado anteriormente, vamos a obtener filas de nuestros juegos mejor data.
df_resultado = pd.merge(df_final, df[['item_id', 'app_name']], on='item_id', how='inner')
df_resultado

,item_id,genres,recommend,playtime_forever,app_name
0,70,53,52,56884,Half-Life
1,70,53,52,56884,Half-Life
2,70,53,52,56884,Half-Life
3,70,53,52,56884,Half-Life
4,70,53,52,56884,Half-Life
...,...,...,...,...,...
35738,80,11,8,37715,Counter-Strike: Condition Zero
35739,80,11,8,37715,Counter-Strike: Condition Zero
35740,80,11,8,37715,Counter-Strike: Condition Zero
35741,80,11,8,37715,Counter-Strike: Condition Zero


In [38]:
# Eliminamos duplicados y nulos
df_resultado = df_resultado.drop_duplicates()
df_resultado = df_resultado.dropna()
df_resultado

,item_id,genres,recommend,playtime_forever,app_name
0,70,53,52,56884,Half-Life
53,2400,22,21,10608,The Ship: Murder Party
75,1520,7,5,17155,DEFCON
82,4000,1533,1491,49464882,Garry's Mod
1615,2990,6,6,5933,FlatOut 2™
...,...,...,...,...,...
35661,1510,5,5,5666,Uplink
35666,20,15,10,72492,Team Fortress Classic
35681,10,45,44,747975,Counter-Strike
35726,13230,6,6,6616,Unreal Tournament 2004: Editor's Choice Edition


In [39]:
# Ordenamos las columnas, para facilitar el futuro proceso de ML
columnas = ['item_id','app_name','recommend','playtime_forever','genres']
df_ok_item_item = df_resultado[columnas]
df_ok_item_item

,item_id,app_name,recommend,playtime_forever,genres
0,70,Half-Life,52,56884,53
53,2400,The Ship: Murder Party,21,10608,22
75,1520,DEFCON,5,17155,7
82,4000,Garry's Mod,1491,49464882,1533
1615,2990,FlatOut 2™,6,5933,6
...,...,...,...,...,...
35661,1510,Uplink,5,5666,5
35666,20,Team Fortress Classic,10,72492,15
35681,10,Counter-Strike,44,747975,45
35726,13230,Unreal Tournament 2004: Editor's Choice Edition,6,6616,6


In [40]:
# Guardamos nuestro df para el proceso de ML recomendaciones
df_ok_item_item.to_csv('dataML_item_item.csv', index=False)

# Filtrar para el proceso de recomendación **usuario-item**

In [42]:
# Cargamos DF
df_us = pd.read_csv('../data/fusionada.csv')

### Trabajamos la columna **'recommend'**

In [43]:
usua_fil = df_us[df_us['recommend'] == True]['user_id'].value_counts().reset_index()
usua_fil.columns = ['user_id', 'frecuencia_rec']
usua_fil

,user_id,frecuencia_rec
0,Deus_VuIt,10
1,76561198072949124,10
2,76561198064484479,10
3,asianturtlefrenzy,10
4,76561198125859026,10
...,...,...
17914,76561198082559637,1
17915,76561198090194644,1
17916,testicularsoundexpress,1
17917,76561198078896873,1


In [44]:
# Aseguramos que la columna tenga datos INT
usua_fil['frecuencia_rec'] = usua_fil['frecuencia_rec'].astype(int)
#Filtrar usuarios con más de 2 interacciones
filtro_rec = usua_fil[usua_fil['frecuencia_rec'] >= 3]
filtro_rec

,user_id,frecuencia_rec
0,Deus_VuIt,10
1,76561198072949124,10
2,76561198064484479,10
3,asianturtlefrenzy,10
4,76561198125859026,10
...,...,...
4381,MMR_Assasin,3
4382,Frozenicypole,3
4383,SwaRIsLoveSwarIsLife,3
4384,leafshinobi,3


In [45]:
# Sumar el tiempo de juego para cada juego (user_id)
us_time = df_us.groupby('user_id')['playtime_forever'].sum().reset_index()
us_time.columns = ['user_id', 'total_playtime']
us_time

,user_id,total_playtime
0,--000--,2949.0
1,--ace--,1269.0
2,--ionex--,36320.0
3,-2SV-vuLB-Kg,31337.0
4,-Azsael-,11889.0
...,...,...
18951,zuzuga2003,72732.0
18952,zv_odd,6452.0
18953,zvanik,38959.0
18954,zynxgameth,3846.0


In [46]:
# Al verificar que nuestra columna total_playtime tiene datos booleanos, los cambiamos a enteros, para optimizar los futuros procesos.
us_time['total_playtime'] = us_time['total_playtime'].astype(int)
#Filtrar juegos con tiempo de juego superior a 4000 minutos
us_time_fil = us_time[us_time['total_playtime'] >= 4000]
us_time_fil

,user_id,total_playtime
2,--ionex--,36320
3,-2SV-vuLB-Kg,31337
4,-Azsael-,11889
6,-Kenny,12842
9,-SEVEN-,187309
...,...,...
18950,zunbae,116985
18951,zuzuga2003,72732
18952,zv_odd,6452
18953,zvanik,38959


In [47]:
df_merged_us = pd.merge(filtro_rec, us_time_fil, on='user_id')
df_merged_us

,user_id,frecuencia_rec,total_playtime
0,Deus_VuIt,10,70022
1,76561198072949124,10,49574
2,76561198064484479,10,33566
3,asianturtlefrenzy,10,7008
4,Findoogle,10,28915
...,...,...,...
3725,Imposs1bru,3,19226
3726,MMR_Assasin,3,83198
3727,Frozenicypole,3,15124
3728,SwaRIsLoveSwarIsLife,3,55829


In [48]:
# Eliminamos duplicados y nulos
df_merged_us = df_merged_us.drop_duplicates()
df_merged_us = df_merged_us.dropna()
df_merged_us

,user_id,frecuencia_rec,total_playtime
0,Deus_VuIt,10,70022
1,76561198072949124,10,49574
2,76561198064484479,10,33566
3,asianturtlefrenzy,10,7008
4,Findoogle,10,28915
...,...,...,...
3725,Imposs1bru,3,19226
3726,MMR_Assasin,3,83198
3727,Frozenicypole,3,15124
3728,SwaRIsLoveSwarIsLife,3,55829


In [50]:
# Mergeamos nuestra data original
df_finalx = pd.merge(df_merged_us, df, on='user_id')
df_finalx

,user_id,frecuencia_rec,total_playtime,app_name,release_date,price,item_id,developer,clasificacion,playtime_forever,items_count,recommend,sentiment_analysis,posted
0,Deus_VuIt,10,70022,Mount & Blade: Warband,2010,19.99,48700,TaleWorlds Entertainment,Action,25589.0,82.0,True,2.0,2015-03-11
1,Deus_VuIt,10,70022,The Lord of the Rings Online™,2012,0.00,212500,"Standing Stone Games, LLC",RPG,10315.0,82.0,True,1.0,2013-11-07
2,Deus_VuIt,10,70022,Counter-Strike: Global Offensive,2012,14.99,730,Valve,Action,21358.0,82.0,True,2.0,2015-04-03
3,Deus_VuIt,10,70022,The Way of Life Free Edition,2016,0.00,310370,"Fabio Ferrara,Davide Caio,Nicolò Azzolini,Anto...",Indie,11.0,82.0,True,1.0,2015-01-12
4,Deus_VuIt,10,70022,ARK: Survival Evolved,2017,59.99,346110,"Studio Wildcard,Instinct Games,Efecto Studios,...",Action,10435.0,82.0,True,1.0,2012-02-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,SwaRIsLoveSwarIsLife,3,55829,Arma 2: Operation Arrowhead,2010,19.99,33930,Bohemia Interactive,Action,12489.0,24.0,True,1.0,2015-05-04
17881,76561198031799936,3,19189,The Binding of Isaac: Rebirth,2014,14.99,250900,"Nicalis, Inc.",Action,14324.0,187.0,True,1.0,2014-11-04
17882,76561198031799936,3,19189,Ben and Ed,2015,9.99,395200,Sluggerfly,Action,623.0,187.0,False,1.0,2012-07-24
17883,76561198031799936,3,19189,Fingered,2015,1.87,384360,"Edmund McMillen,James Id",Casual,102.0,187.0,True,2.0,2015-08-22


In [52]:
# Selección de las columnas para el siguiente proceso de ML
columnas = ['app_name', 'item_id', 'playtime_forever', 'user_id','recommend', 'sentiment_analysis']
df_final_users = df_finalx[columnas]
df_final_users

,app_name,item_id,playtime_forever,user_id,recommend,sentiment_analysis
0,Mount & Blade: Warband,48700,25589.0,Deus_VuIt,True,2.0
1,The Lord of the Rings Online™,212500,10315.0,Deus_VuIt,True,1.0
2,Counter-Strike: Global Offensive,730,21358.0,Deus_VuIt,True,2.0
3,The Way of Life Free Edition,310370,11.0,Deus_VuIt,True,1.0
4,ARK: Survival Evolved,346110,10435.0,Deus_VuIt,True,1.0
...,...,...,...,...,...,...
17880,Arma 2: Operation Arrowhead,33930,12489.0,SwaRIsLoveSwarIsLife,True,1.0
17881,The Binding of Isaac: Rebirth,250900,14324.0,76561198031799936,True,1.0
17882,Ben and Ed,395200,623.0,76561198031799936,False,1.0
17883,Fingered,384360,102.0,76561198031799936,True,2.0


In [53]:
# Guardamos nuestro df para el proceso de ML recomendaciones
df_final_users.to_csv('dataML_user_item.csv', index=False)